<a href="https://colab.research.google.com/github/ValentinaBykova/neuron/blob/master/lab8/Lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторна робота №8
## студентки КН-31, Бикової Валентини
## Тема: Творче завдання

## Нейронна мережа Елмана

In [1]:
from typing import Union, List

import numpy as np


def sigmoid(x: np.ndarray) -> np.ndarray:
    """
    Sigmoid activation function

    :param x: input matrix
    :return: resulted matrix
    """

    return 1 / (1 + np.exp(-x))


def softmax(x: Union[np.ndarray, List]) -> np.ndarray:
    """
    Softmax activation function

    :param x: input matrix
    :return: resulted matrix
    """
    e_x = np.exp(x - np.max(x))

    return e_x / e_x.sum(axis=0)


def softmax_der(x: Union[np.ndarray, List]) -> np.ndarray:
    """
    Derivative of softmax activation function

    :param x: input matrix
    :return: resulted matrix
    """
    res = softmax(x) * (1 - softmax(x))

    return res


def sigmoid_der(x: np.ndarray) -> np.ndarray:
    """
    Derivative of sigmoid activation function

    :param x: input matrix
    :return: resulted matrix
    """

    return sigmoid(x) * (1 - sigmoid(x))


def log(x: np.ndarray) -> np.ndarray:
    """
    Log activation function (natural algorithm)

    :url: http://jmlda.org/papers/doc/2011/no1/Rudoy2011Selection.pdf#page=12
    :param x: input matrix
    :return: resulted matrix
    """

    res = np.log(x + np.sqrt(x**2 + 1))
    res[x > 74.2] = 5
    res[x < -74.2] = -5
    res = res / 5

    return res


def log_der(x: np.ndarray) -> np.ndarray:
    """
    Derivative of log activation function (natural algorithm)

    :param x: input matrix
    :return: resulted matrix
    """

    res = 1 / (np.sqrt(x**2 + 1))
    res[x > 74.2] = 0
    res[x < -74.2] = 0
    res = res / 5

    return res


def linear(x: np.ndarray) -> np.ndarray:
    """
    Linear activation function (natural algorithm)

    :url: http://jmlda.org/papers/doc/2011/no1/Rudoy2011Selection.pdf#page=12
    :param x: input matrix
    :return: resulted matrix
    """

    return x


def linear_der(x: np.ndarray) -> Union[np.ndarray, int]:
    """
    Derivative of log activation function (natural algorithm)

    :param x: input matrix
    :return: resulted matrix
    """

    return 1


def cross_entropy_loss(y_pred: Union[np.ndarray, List],
                       y_true: Union[np.ndarray, List]) -> List[Union[float, np.ndarray]]:
    """
    Cross entropy loss.
    Use it after sigmoid function!

    :param y_pred: values predicted by NN (with softmax on top of it)
    :param y_true: true value
    :return: loss value
    """
    y_pred = np.array(y_pred, dtype=np.float16)

    y_true = np.array(y_true, dtype=np.float16)
    y_true_argmax = y_true.argmax(axis=0)

    y_pred[y_true_argmax] = np.clip(y_pred[y_true_argmax], a_min=0.0001, a_max=None)

    log_likelihood = - np.log(y_pred[y_true_argmax])
    loss = np.sum(log_likelihood)

    return loss


def cross_entropy_loss_der(y_pred: Union[np.ndarray, List],
                           y_true: Union[np.ndarray, List]) -> Union[List[float], np.ndarray]:
    """
    Cross entropy loss derivative.
    Use it after sigmoid function!

    :param y_pred: values predicted by NN (with softmax, sigmoid on top of it)
    :param y_true: true value
    :return: loss value
    """
    y_pred = np.array(y_pred, dtype=np.float16)
    grad = linear(x=y_pred)

    y_true = np.array(y_true, dtype=np.float16)
    y_true_argmax = y_true.argmax(axis=0)

    grad[y_true_argmax] = np.clip(grad[y_true_argmax], a_min=0.0001, a_max=None)
    grad[y_true_argmax] -= 1

    step = - grad

    return step


def mse_loss(y_pred: Union[np.ndarray, List],
             y_true: Union[np.ndarray, List]) -> List[Union[float, np.ndarray]]:
    """
    Mean squared error loss

    :param y_pred: values predicted by NN (WITHOUT softmax, sigmoid on top of it)
    :param y_true: true value
    :return: loss value
    """

    mse_value = ((y_pred - y_true) ** 2).mean()

    return mse_value


def mse_loss_der(y_pred: Union[np.ndarray, List],
                 y_true: Union[np.ndarray, List]) -> List[Union[float, np.ndarray]]:
    """
    Mean squared error loss derivative

    :param y_pred: values predicted by NN (WITHOUT softmax, sigmoid on top of it)
    :param y_true: true value
    :return: loss value
    """

    mse_der_value = - 2 * (y_pred - y_true)

    return mse_der_value

In [2]:
from typing import List, Union

import numpy as np


class Jordan:
    """Jordan network"""

    def __init__(self, lr: float,
                 momentum: float,
                 shape: List[int], make_zero_context: bool = False):
        self.lr = lr
        self.momentum = momentum
        self.make_zero_context = make_zero_context

        self.shape = shape

        self.n_layers = len(shape)

        self.layers = self.__init_layers__()
        self.weights = self.__init_weights__()

        self.dw = [0] * len(self.weights)

    def __init_layers__(self) -> List[np.ndarray]:
        """
        Initialize layers of NN

        :return: list of initialized layers
        """

        layers = list()

        layers.append(np.ones(self.shape[0] + self.shape[-1] + 1))

        for i in range(1, self.n_layers):
            layers.append(np.ones(self.shape[i]))

        return layers

    def __init_weights__(self) -> List[np.ndarray]:
        """
        Neural networks he initialization

        :url: https://medium.com/@prateekvishnu/xavier-and-he-normal-he-et-al-initialization-8e3d7a087528
        :return: list of weights
        """

        if len(self.layers) == 0:
            raise ValueError('Before weight initialization, initialize layers!')

        weights = list()

        for i in range(self.n_layers - 1):
            curr_weights = np.random.randn(self.layers[i].size,
                                           self.layers[i + 1].size) * np.sqrt(2 / self.layers[i].size)

            weights.append(curr_weights)

        return weights

    def propagate_forward(self, x: Union[np.ndarray, List]) -> np.ndarray:
        """
        Propagate data in network forward. Forward pass

        :param x: data to propagate
        :return: result of neural network
        """

        self.layers[0][0: self.shape[0]] = x

        if self.make_zero_context:
            self.layers[0][self.shape[0]: -1] = np.zeros_like(self.layers[-1])
        else:
            self.layers[0][self.shape[0]: -1] = self.layers[-1]

        for i in range(1, len(self.shape) - 1):
            self.layers[i][...] = linear(
                np.dot(self.layers[i - 1], self.weights[i - 1])
            )

        if len(self.shape) - 2 >= 0:
            last_idx = len(self.shape) - 1

            self.layers[last_idx][...] = linear(
                np.dot(self.layers[last_idx - 1], self.weights[last_idx - 1])
            )

        return self.layers[-1]

    def propagate_backward(self, target) -> float:
        """
        Performs backpropagation on neural network.

        :param target: desired result
        :return: error of the network
        """

        deltas = list()

        loss_number = mse_loss(y_pred=self.layers[-1],
                               y_true=target)
        last_layer_delta = mse_loss_der(y_pred=self.layers[-1],
                                        y_true=target)

        deltas.append(last_layer_delta)

        for i in range(len(self.shape) - 2, 0, -1):
            curr_delta = np.dot(deltas[0],
                                self.weights[i].T * linear_der(self.layers[i]))

            deltas.insert(0, curr_delta)

        for i in range(len(self.weights)):
            layer = np.atleast_2d(self.layers[i])
            curr_delta = np.atleast_2d(deltas[i])

            curr_dw = np.dot(layer.T, curr_delta)

            self.weights[i] += self.lr * curr_dw + self.lr * self.momentum * self.dw[i]

            self.dw[i] = curr_dw

        return loss_number


if __name__ == '__main__':
    network = Jordan(lr=0.01, momentum=0.1, shape=[10, 15, 15, 1])

    x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    result = network.propagate_forward(x=x)
    error = network.propagate_backward(target=1, )

    print(f'Result: {result}')
    print(f'Error: {error}')

Result: [11.51516912]
Error: 110.56878157022702


## Висновики
#### Під час лабораторної роботи я вивчила новий для себе вид нейронних мереж - мережа Елмана та розглянула її застосування.